In [1]:
import torch
import yaml
from tqdm import tqdm
import numpy as np
from data import BPRSampleGenerator, SeqBPRDataset
from torch.utils.data import DataLoader
from model import SeqLearn

In [2]:
with open("/graduation_design/bpr/config/bpr.yaml", 'r', encoding='utf-8') as f:
    args = yaml.unsafe_load(f)
args

{'base_path': '/graduation_design/',
 'topk': 10,
 'data': {'train_valid_split': 0.95,
  'maxlen': 30,
  'name': 'ml-1m',
  'sep': '::',
  'item_path': '/graduation_design/data/ml-1m/movies.dat',
  'item_emb_path': '/graduation_design/data/ml-1m/item_embeddings.npy',
  'path': '/graduation_design/data/ml-1m/ratings.dat',
  'num_negatives': 1,
  'user_threshold': 10,
  'item_threshold': 10,
  'rating_threshold': 2,
  'base_model': ['acf', 'fdsa', 'harnn', 'caser', 'pfmc', 'sasrec', 'anam'],
  'base_model_path': '/graduation_design/base_model_results/ml-1m'},
 'model': {'lr': 0.001,
  'type': 'SASEM',
  'lamda': 1e-05,
  'hidden_dim': 32,
  'device': 'cuda:0',
  'optimizer': 'AdamOptimizer',
  'tradeoff': 2,
  'div_module': 'cov',
  'pretrain_llm': 'bert-base-uncased'},
 'epoch': 1,
 'batch_size': 512}

In [9]:
acf = np.load(args['data']['base_model_path'] + f"/acf.npy")

FileNotFoundError: [Errno 2] No such file or directory: '/graduation_design/base_model_results/ml-1m/acf.npy'

In [3]:
# 创建数据生成器
generator = BPRSampleGenerator(args['data'])
seq_samples = generator.generate_seq_samples(
    seq_len=args['data']['maxlen'],
    num_negatives=args['data']['num_negatives']
)

# 创建数据集
dataset = SeqBPRDataset(seq_samples, args['model']['device'])
train_size = int(args['data']['train_valid_split'] * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=args['batch_size'], shuffle=True)

>>>> 交互索引范围: 最小值 = 0, 最大值 = 834448
>>>> 数据加载完成: 834449 条交互, 6033 个用户, 3123 个物品
>>>> 基模型的预测结果加载完成: (834449, 7, 102)
>>>> 构建了 6033 个用户的历史交互序列


>>>> 生成序列样本: 100%|██████████| 6033/6033 [00:02<00:00, 2081.01it/s]


>>>> 生成了 828416 个序列感知样本


In [5]:
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, drop_last=True)

In [6]:
model = np.load(args['data']['base_model_path'] + f"/sasrec.npy")

with torch.no_grad():
    ndcg_scores = []
    for batch in tqdm(test_loader, desc="计算NDCG@10..."):
        users, user_seq, pos_items, neg_items, base_model_preds = batch
        
        # 遍历batch中的每个样本
        for i in range(len(users)):
            # 获取交互索引
            user_id = users[i].item()
            pos_item_id = pos_items[i].item()
            interaction_idx = generator.get_interaction_index(user_id, pos_item_id)
            if interaction_idx is None:
                continue

            # 获取模型推荐的top物品列表
            top_items = model[interaction_idx][2:2+args['topk']]

            # 获取用户的实际交互物品
            true_items = generator.user_interacted_items[user_id]

            # 计算DCG
            dcg = 0
            for j, item_idx in enumerate(top_items):
                if item_idx in true_items:
                    dcg += 1 / np.log2(j + 2)
            
            # 计算IDCG
            idcg = 0
            for j in range(min(len(true_items), args['topk'])):
                idcg += 1 / np.log2(j + 2)

            # 计算NDCG
            ndcg = dcg / idcg if idcg > 0 else 0
            ndcg_scores.append(ndcg)

np.mean(ndcg_scores)

计算NDCG@10...: 100%|██████████| 41421/41421 [00:15<00:00, 2673.83it/s]


0.219812818637405

## 集成模型预测结果

In [4]:
test_loader = DataLoader(test_dataset, batch_size=args['batch_size'], shuffle=False, drop_last=True)

model = SeqLearn(args['model'], args['data'], 6033, generator.n_item)
# 加载checkpoint
ckpt = torch.load(f"/root/autodl-tmp/score_ckpt/score_epoch1_batch900.pth")

# 过滤掉不需要加载的层
filtered_ckpt = {k: v for k, v in ckpt.items() if not k.startswith('item_tower.cex')}

# 加载过滤后的权重
model.load_state_dict(filtered_ckpt, strict=False)

model.eval()

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


>>>> 加载预计算的物品嵌入...


SeqLearn(
  (cem): ContentExtractionModule(
    (llm): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(50265, 1024, padding_idx=1)
        (position_embeddings): Embedding(514, 1024, padding_idx=1)
        (token_type_embeddings): Embedding(1, 1024)
        (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-23): 24 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSdpaSelfAttention(
                (query): Linear(in_features=1024, out_features=1024, bias=True)
                (key): Linear(in_features=1024, out_features=1024, bias=True)
                (value): Linear(in_features=1024, out_features=1024, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): RobertaSelfOutput(
                (dense): Linear

In [5]:
# 获取一个批次的数据
batch = next(iter(test_loader))
users, user_seq, items, scores, base_model_preds = batch

print("用户张量形状:", users.shape)
print("用户序列张量形状:", user_seq.shape) 
print("物品张量形状:", items.shape)
print("分数张量形状:", scores.shape)
print("基础模型预测张量形状:", base_model_preds.shape if base_model_preds is not None else None)

用户张量形状: torch.Size([512])
用户序列张量形状: torch.Size([512, 30])
物品张量形状: torch.Size([512])
分数张量形状: torch.Size([512])
基础模型预测张量形状: torch.Size([512, 7, 100])


In [8]:
user_idx = 0
user = users[user_idx:user_idx+1]
user_seq_i = user_seq[user_idx:user_idx+1]
base_model_preds_i = base_model_preds[user_idx:user_idx+1] if base_model_preds is not None else None

# 获取所有物品ID
all_item_ids = torch.arange(generator.n_item, device=args['model']['device'])

all_scores = []

for i in range(0, len(all_item_ids), args['batch_size']):
    batch_items = all_item_ids[i:i + args['batch_size']]
    cnt = len(batch_items)

    user_repeated = user.repeat(cnt)
    user_seq_repeated = user_seq_i.repeat(cnt, 1)
    base_model_preds_repeated = base_model_preds_i.repeat(cnt, 1, 1)
    # 预测分数
    print(user_repeated.shape, user_seq_repeated.shape, batch_items.shape, base_model_preds_repeated.shape)
    batch_scores = model(user_repeated, user_seq_repeated, batch_items, base_model_preds_repeated)
    all_scores.append(batch_scores)

torch.Size([512]) torch.Size([512, 30]) torch.Size([512]) torch.Size([512, 7, 100])
torch.Size([512]) torch.Size([512, 30]) torch.Size([512]) torch.Size([512, 7, 100])
torch.Size([512]) torch.Size([512, 30]) torch.Size([512]) torch.Size([512, 7, 100])
torch.Size([512]) torch.Size([512, 30]) torch.Size([512]) torch.Size([512, 7, 100])
torch.Size([512]) torch.Size([512, 30]) torch.Size([512]) torch.Size([512, 7, 100])
torch.Size([512]) torch.Size([512, 30]) torch.Size([512]) torch.Size([512, 7, 100])
torch.Size([51]) torch.Size([51, 30]) torch.Size([51]) torch.Size([51, 7, 100])


In [15]:
user_scores = torch.cat(all_scores, dim=0).squeeze()
print(f"用户{user.item()}的预测分数范围: {user_scores.min().item()} ~ {user_scores.max().item()}")

用户779的预测分数范围: 3.9388139247894287 ~ 3.9545481204986572


In [ ]:
with torch.no_grad():
    for batch in tqdm(test_loader, desc="计算测试集NDCG"):
        users, user_seq, items, scores, base_model_preds = batch

        # 对每个用户计算NDCG
        for user_idx in range(len(users)):
            # 获取当前用户
            user = users[user_idx:user_idx+1]
            user_seq_i = user_seq[user_idx:user_idx+1]
            base_model_preds_i = base_model_preds[user_idx:user_idx+1] if base_model_preds is not None else None

            # 获取所有物品ID
            all_item_ids = torch.arange(generator.n_item, device=args['model']['device'])

            all_scores = []

            for i in range(0, len(all_item_ids), args['batch_size']):
                batch_items = all_item_ids[i:i + args['batch_size']]
                cnt = len(batch_items)

                user_repeated = user.repeat(cnt)
                user_seq_repeated = user_seq_i.repeat(cnt, 1)
                base_model_preds_repeated = base_model_preds_i.repeat(cnt, 1, 1)
                # 预测分数
                batch_scores = model(user_repeated, user_seq_repeated, batch_items, base_model_preds_repeated)
                all_scores.append(batch_scores)

            # 合并所有批次的分数
            user_scores = torch.cat(all_scores, dim=0).squeeze()
            print(user_scores.shape)

            # 获取前k个物品
            _, indices = torch.topk(user_scores, args['topk'])

            # 获取用户的实际交互物品
            true_items = generator.user_interacted_items[user.item()]

            # 计算DCG
            dcg = 0
            for i, item_idx in enumerate(indices):
                if item_idx.item() in true_items:
                    dcg += 1 / np.log2(i + 2)

            # 计算IDCG
            idcg = 0
            for i in range(min(len(true_items), args['topk'])):
                idcg += 1 / np.log2(i + 2)

            # 计算NDCG
            ndcg = dcg / idcg if idcg > 0 else 0
            ndcg_scores.append(ndcg)

np.mean(ndcg_scores)